In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [249]:
df = pd.read_excel('population_pol.xlsx')
df = df[['Charakter miejscowości', 'Płeć', 'Wiek - pojedyncze roczniki', 2021, 2022, 2023]]
df.rename(columns={'Charakter miejscowości': 'locality',
                   'Płeć': 'gender',
                   'Wiek - pojedyncze roczniki': 'age',
                   2021: '2021',
                   2022: '2022',
                   2023: '2023'},
          inplace=True)
df.replace({'Ogółem': 'total',
            'Mężczyźni': 'men',
            'Kobiety': 'women',
            'Wieś': 'village',
            'Miasto': 'city'}, 
           inplace=True)

In [250]:
df

,locality,gender,age,2021,2022,2023
0,total,total,total,37907704,37766327,37636508
1,total,total,0 lat,328809,299315,266966
2,total,total,1 rok,349581,334114,304379
3,total,total,2 lata,374820,349957,334484
4,total,total,3 lata,392977,375326,350399
...,...,...,...,...,...,...
913,village,men,96 lat,829,843,913
914,village,men,97 lat,559,553,613
915,village,men,98 lat,416,387,388
916,village,men,99 lat,307,292,282


In [213]:
mask_total_women = ((df['gender'] == 'women') & (df['age'] == 'total') & (df['locality'] == 'total'))
total_woman = df[mask_total_women]#[['2021', '2022', '2023']].values

mask_total_men = ((df['gender'] == 'men') & (df['age'] == 'total') & (df['locality'] == 'total'))
total_men = df[mask_total_men]#[['2021', '2022', '2023']].values

In [222]:
gender_mask = ((df['age'] == 'total') & (df['locality'] == 'total') & (df['gender'] != 'total') )
gender_count = df[gender_mask][['gender', '2021', '2022', '2023']]
gender_count

,gender,2021,2022,2023
102,women,19584787,19517006,19454109
204,men,18322917,18249321,18182399


In [223]:
gender_count.to_csv('gender_structure.csv')

In [ ]:
years_indices = gender_count.melt(id_vars=['interval'], var_name='year', value_name='count')
intervals_columns = years_indices.pivot(index='year', columns='interval', values='count')

In [246]:
melted = gender_count.melt(id_vars=['gender'], var_name='year', value_name='count')
pivoted = melted.pivot(index='year', columns='gender', values='count')

In [247]:
pivoted.to_csv('gender_structure.csv')


In [251]:
df['age'] = df['age'].str.extract(r'(\d+)')
df[(~df['age'].isna()) & (df['age'] != 'total')]['age'] = df[(~df['age'].isna()) & (df['age'] != 'total')]['age'].astype(int)

C:\Users\wypyc\AppData\Local\Temp\ipykernel_22432\725831280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(~df['age'].isna()) & (df['age'] != 'total')]['age'] = df[(~df['age'].isna()) & (df['age'] != 'total')]['age'].astype(int)


In [165]:
age_mask = (~df['age'].isna()) & (df['age'] != 'total') & (df['locality'] == 'total') & (df['gender'] == 'total')
age_df = df[age_mask]
age_df['age'] = age_df['age'].astype(int)

C:\Users\wypyc\AppData\Local\Temp\ipykernel_22432\719661220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df['age'] = age_df['age'].astype(int)


In [201]:
ending_age = [6, 12, 18, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, 100]
age_structure = pd.DataFrame(columns=['interval', '2021', '2022', '2023'])
for i, end in enumerate(ending_age):
    if i == 0:
        pop_sum = age_df[(age_df['age'] >= 0) & (age_df['age'] <= end)][['2021', '2022', '2023']].sum().to_list()
        #age_structure.append((pop_sum, 0, end))
        pop_sum.insert(0, f'0-{end}')        
    else:
        pop_sum = age_df[(age_df['age'] >= ending_age[i-1] + 1) & (age_df['age'] <= end)][['2021', '2022', '2023']].sum().to_list()
        #age_structure.append((pop_sum, ending_age[i-1], end))
        if i == len(ending_age) - 1:
            pop_sum.insert(0, f'{ending_age[i-1]+1}+')
        else:
            pop_sum.insert(0, f'{ending_age[i-1]+1}-{end}')
    new_row = pd.DataFrame([pop_sum], columns=age_structure.columns)
    age_structure = pd.concat([age_structure, new_row])

In [203]:
age_structure.to_csv('age_structure.csv')

In [234]:
age_structure

,interval,2021,2022,2023
0,0-6,2640713,2560544,2438726
0,7-12,2441104,2398469,2369027
0,13-18,2254232,2341670,2418796
0,19-24,2230589,2171202,2135906
0,25-29,2274032,2176867,2083051
0,30-34,2732843,2657137,2573966
0,35-39,3188086,3103931,2990800
0,40-44,3082559,3121806,3177592
0,45-49,2785620,2864415,2929547
0,50-54,2303011,2364564,2443343


In [235]:
years_indices = age_structure.melt(id_vars=['interval'], var_name='year', value_name='count')

In [236]:
intervals_columns = years_indices.pivot(index='year', columns='interval', values='count')

In [240]:
intervals_columns = intervals_columns[['0-6', '7-12', '13-18', '19-24', '25-29', '30-34', '35-39', '40-44', '45-49',
       '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84',
       '85+']]

In [241]:
intervals_columns

interval,0-6,7-12,13-18,19-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
year,,,,,,,,,,,,,,,,,
2021,2640713,2441104,2254232,2230589,2274032,2732843,3188086,3082559,2785620,2303011,2244838,2554935,2494976,1985625,1072017,829368,793156
2022,2560544,2398469,2341670,2171202,2176867,2657137,3103931,3121806,2864415,2364564,2208012,2444255,2513554,2053284,1186305,801434,798878
2023,2438726,2369027,2418796,2135906,2083051,2573966,2990800,3177592,2929547,2443343,2182033,2343827,2519590,2108909,1320176,784361,816858


In [242]:
intervals_columns.to_csv('age_structure.csv')


In [264]:
mask13 = (df['locality'] == 'total') & (df['gender'] == 'total') & (df['age'].astype(float) >= 13)
df[mask13]['2021'].sum()


np.int64(32825887)